# simple transformer

In [ ]:
#mounting Google drive
import sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df  = pd.read_csv('/content/drive/MyDrive/mlprep/Final_Dataset.csv', error_bad_lines=False)

<ipython-input-2-ca9094458ed4>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df  = pd.read_csv('/content/drive/MyDrive/mlprep/Final_Dataset.csv', error_bad_lines=False)


In [ ]:
train_df = pd.DataFrame(columns=['premises', 'Text', 'label'])
test_df = pd.DataFrame(columns=['premises', 'Text', 'label'])

In [ ]:
j =0
for i in range(len(df)):
    j = i%317
    df2 = pd.DataFrame({'premises': [df.iloc[i,0]], 'Text': [df.iloc[i,1]], 'label': [df.iloc[i,2]]})
    if j>=0 and j<250:
        train_df = pd.concat([train_df, df2], ignore_index=True)
    else:
        test_df = pd.concat([test_df, df2], ignore_index=True)

In [ ]:
# Shuffle the DataFrame
train_df = train_df.sample(frac=1, random_state=42)  # Set random_state for reproducibility

# Reset the index to make it continuous
train_df = train_df.reset_index(drop=True)

# Shuffle the DataFrame
test_df = test_df.sample(frac=1, random_state=42)  # Set random_state for reproducibility

# Reset the index to make it continuous
test_df = test_df.reset_index(drop=True)

In [ ]:
eval_df = test_df.copy()

In [ ]:
train_df = train_df.rename(columns={'premises': 'text_a', 'Text': 'text_b', 'label': 'labels'}).dropna()
eval_df = eval_df.rename(columns={'premises': 'text_a', 'Text': 'text_b', 'label': 'labels'}).dropna()

In [ ]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text_a  804 non-null    object
 1   text_b  804 non-null    object
 2   labels  804 non-null    object
dtypes: object(3)
memory usage: 19.0+ KB


In [ ]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'evaluate_during_training': True,
    'max_seq_length': 512,
    'num_train_epochs': 10,
    'evaluate_during_training_steps': 50,
    'train_batch_size': 16,
    'regression': False,
}

In [ ]:
%pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 27.4 MB/s eta 0:00:00


In [ ]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=train_args)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from scipy.stats import pearsonr, spearmanr


def pearson_corr(preds, labels):
    return pearsonr(preds, labels)[0]


def spearman_corr(preds, labels):
    return spearmanr(preds, labels)[0]

In [ ]:
# Train the model
model.train_model(train_df, eval_df=eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)

  0%|          | 0/3000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/804 [00:00<?, ?it/s]

  0%|          | 0/804 [00:00<?, ?it/s]

...

  0%|          | 0/804 [00:00<?, ?it/s]

  0%|          | 0/804 [00:00<?, ?it/s]

  0%|          | 0/804 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/188 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# model.train_model(train_df)

In [ ]:
model.model.save_pretrained('model1')
model.tokenizer.save_pretrained('model1')
model.config.save_pretrained('model1/')

In [ ]:
model = ClassificationModel('roberta','model1')

In [ ]:
predictions, raw_outputs = model.predict([["The man is sleeping with his wife.", "The man is sleeping"]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions

array([1])

In [ ]:
raw_outputs

array([[-5.5       ,  5.47265625]])

In [ ]:
predictions, raw_outputs = model.predict([["animal drinks water", "gvhghgv."]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions

array([1])

In [ ]:
raw_outputs

array([[-5.5703125 ,  5.62109375]])

In [ ]:
from google.colab import files
files.download('model1')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('outputs')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('runs')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('cache_dir')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/model1')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>